### 1) Write a Python script that fetches holiday data and saves it as a csv file named "holidays.csv".​ To obtain the data, you can either screen scrape the Time and Date​ site, or parse the API (free trial​ account required).

In [118]:
import pandas as pd
import numpy as np
import requests
import lxml.html
import datetime

In [15]:
page = requests.get("https://www.timeanddate.com/holidays/us/")

In [16]:
tree = lxml.html.fromstring(page.text)

In [17]:
dates = tree.xpath("//th/text()")[5:]
len(dates)

402

In [18]:
days = tree.xpath("//tr/td[1]/text()")[:402]
len(days)

402

In [19]:
holiday_names = tree.xpath("//td/a/text()")
len(holiday_names)

402

In [20]:
holiday_type = tree.xpath("//tr/td[3]/text()")
len(holiday_type)

402

In [21]:
t = tree.xpath("//tr/td[4]")
details = []
for elem in t:
    val = elem.xpath('text()')
    temp = ''
    temp1 = ''
    if(len(val)>1):
        if(val[0]=='All except '):
            temp1 = temp1 + 'All except '
            
            for x in range(1,len(val)+1):
                tt = elem.xpath("span["+str(x)+"]/text()")
                temp1 = temp1 + tt[0] + ','
            details.append([temp1])
        else:
            ff = elem.xpath("span/text()")
            for x in ff:
                temp = temp+x + ','
            details.append([temp])       
    else:
        details.append(val)
len(details)

402

In [22]:
holidays_df = pd.DataFrame({'Date': dates,
                            'Day': days,
                            'Name':holiday_names,
                            'Type':holiday_type,
                            'Details': details})

In [23]:
holidays_df['Details'] = holidays_df['Details'].apply(lambda x: ','.join(map(str, x)))

In [24]:
holidays_df

,Date,Day,Name,Type,Details
0,Jan 1,Tuesday,New Year's Day,Federal Holiday,
1,Jan 4,Friday,World Braille Day,Worldwide observance,
2,Jan 6,Sunday,Epiphany,Christian,
3,Jan 7,Monday,Orthodox Christmas Day,Orthodox,
4,Jan 7,Monday,International Programmers' Day,Worldwide observance,
5,Jan 13,Sunday,Stephen Foster Memorial Day,Observance,
6,Jan 14,Monday,Orthodox New Year,Orthodox,
7,Jan 18,Friday,Lee-Jackson Day,State holiday,Virginia
8,Jan 19,Saturday,Robert E. Lee's Birthday,State holiday,Florida
9,Jan 19,Saturday,Confederate Heroes' Day,State holiday,Texas


In [ ]:
holidays_df.to_csv("holidays.csv")

### 2) Using the csv file from 1), write a​ Python​ function that returns a list of dictionaries of the next 10 upcoming U.S. holidays and dates. 

E.g.


def holidays​():​

    pass



holidays​()​

[
    {
        "name​": "Bastille Day​",
        "date": "Sunday, July 14, 2019",
        "type": "Observance",
        "details​": ""
    },
    ...
]



In [128]:
holidays = pd.read_csv("holidays.csv")
holidays.head(10)

,Date,Day,Name,Type,Details
0,1-Jan,Tuesday,New Year's Day,Federal Holiday,NaN
1,4-Jan,Friday,World Braille Day,Worldwide observance,NaN
2,6-Jan,Sunday,Epiphany,Christian,NaN
3,7-Jan,Monday,Orthodox Christmas Day,Orthodox,NaN
4,7-Jan,Monday,International Programmers' Day,Worldwide observance,NaN
5,13-Jan,Sunday,Stephen Foster Memorial Day,Observance,NaN
6,14-Jan,Monday,Orthodox New Year,Orthodox,NaN
7,18-Jan,Friday,Lee-Jackson Day,State holiday,Virginia
8,19-Jan,Saturday,Robert E. Lee's Birthday,State holiday,Florida
9,19-Jan,Saturday,Confederate Heroes' Day,State holiday,Texas


In [129]:
holidays.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 402 entries, 0 to 401
Data columns (total 5 columns):
Date       402 non-null object
Day        402 non-null object
Name       402 non-null object
Type       402 non-null object
Details    79 non-null object
dtypes: object(5)
memory usage: 15.8+ KB


In [130]:
holidays["Date"] = holidays["Date"].str.replace("-"," ")

In [131]:
holidays["Year"] = 2019

In [132]:
new = holidays["Date"].str.split(" ", expand = True)
holidays["Dates"]= new[0] 
holidays["Month"]= new[1]
holidays.drop(columns =["Date"], inplace = True)

In [133]:
holidays.head()

,Day,Name,Type,Details,Year,Dates,Month
0,Tuesday,New Year's Day,Federal Holiday,NaN,2019,1,Jan
1,Friday,World Braille Day,Worldwide observance,NaN,2019,4,Jan
2,Sunday,Epiphany,Christian,NaN,2019,6,Jan
3,Monday,Orthodox Christmas Day,Orthodox,NaN,2019,7,Jan
4,Monday,International Programmers' Day,Worldwide observance,NaN,2019,7,Jan


In [134]:
holidays['FullDate'] = holidays['Dates'].map(str) + '-' + holidays['Month'].map(str) + '-' + holidays['Year'].map(str)
holidays['FullDate']= pd.to_datetime(holidays['FullDate'])
holidays.head()

,Day,Name,Type,Details,Year,Dates,Month,FullDate
0,Tuesday,New Year's Day,Federal Holiday,NaN,2019,1,Jan,2019-01-01
1,Friday,World Braille Day,Worldwide observance,NaN,2019,4,Jan,2019-01-04
2,Sunday,Epiphany,Christian,NaN,2019,6,Jan,2019-01-06
3,Monday,Orthodox Christmas Day,Orthodox,NaN,2019,7,Jan,2019-01-07
4,Monday,International Programmers' Day,Worldwide observance,NaN,2019,7,Jan,2019-01-07


In [135]:
dict_holidays = holidays.to_dict('records')
dict_holidays

[{'Day': 'Tuesday',
  'Name': "New Year's Day",
  'Type': 'Federal Holiday',
  'Details': nan,
  'Year': 2019,
  'Dates': '1',
  'Month': 'Jan',
  'FullDate': Timestamp('2019-01-01 00:00:00')},
 {'Day': 'Friday',
  'Name': 'World Braille Day',
  'Type': 'Worldwide observance',
  'Details': nan,
  'Year': 2019,
  'Dates': '4',
  'Month': 'Jan',
  'FullDate': Timestamp('2019-01-04 00:00:00')},
 {'Day': 'Sunday',
  'Name': 'Epiphany',
  'Type': 'Christian',
  'Details': nan,
  'Year': 2019,
  'Dates': '6',
  'Month': 'Jan',
  'FullDate': Timestamp('2019-01-06 00:00:00')},
 {'Day': 'Monday',
  'Name': 'Orthodox Christmas Day',
  'Type': 'Orthodox',
  'Details': nan,
  'Year': 2019,
  'Dates': '7',
  'Month': 'Jan',
  'FullDate': Timestamp('2019-01-07 00:00:00')},
 {'Day': 'Monday',
  'Name': "International Programmers' Day",
  'Type': 'Worldwide observance',
  'Details': nan,
  'Year': 2019,
  'Dates': '7',
  'Month': 'Jan',
  'FullDate': Timestamp('2019-01-07 00:00:00')},
 {'Day': 'Sunday

In [213]:
def holiday_10():
    hols_list = []
    curr_date = datetime.datetime.today()
    print("List Of Holidays:\n")
    for hols in dict_holidays:
        if(curr_date <= hols['FullDate']):
            hols_list.append(hols)
              
    sorted_hols = sorted(hols_list, key=lambda x: x['FullDate'])
    print("\n")
    print(sorted_hols[:10]) 
        
holiday_10()

List Of Holidays:



[{'Day': 'Monday', 'Name': 'World Youth Skills Day', 'Type': 'United Nations observance', 'Details': nan, 'Year': 2019, 'Dates': '15', 'Month': 'Jul', 'FullDate': Timestamp('2019-07-15 00:00:00')}, {'Day': 'Thursday', 'Name': 'Nelson Mandela Day', 'Type': 'United Nations observance', 'Details': nan, 'Year': 2019, 'Dates': '18', 'Month': 'Jul', 'FullDate': Timestamp('2019-07-18 00:00:00')}, {'Day': 'Wednesday', 'Name': 'Pioneer Day', 'Type': 'State holiday', 'Details': 'Utah', 'Year': 2019, 'Dates': '24', 'Month': 'Jul', 'FullDate': Timestamp('2019-07-24 00:00:00')}, {'Day': 'Saturday', 'Name': 'National Korean War Veterans Armistice Day', 'Type': 'Observance', 'Details': nan, 'Year': 2019, 'Dates': '27', 'Month': 'Jul', 'FullDate': Timestamp('2019-07-27 00:00:00')}, {'Day': 'Sunday', 'Name': 'World Hepatitis Day', 'Type': 'United Nations observance', 'Details': nan, 'Year': 2019, 'Dates': '28', 'Month': 'Jul', 'FullDate': Timestamp('2019-07-28 00:00:00')}, {'Day': 

### 3) Add a keyword argument "holidayType​"​ to the function from 2) such that it only returns holidays matching the given type, 

e.g., 


def holidays​(holidayType​​=None​):​

    pass

holidays​​(holidayType​​​="federal")    

[
   {
       "name​": "Labor​ Day​",
       "date": "Monday​, September 2, 2019",
       "type": "Federal Holiday​",
       "details​": ""
   },
   ...
]

In [222]:
def holiday_type_func(holiday_type):
    hols_list = []
    curr_date = datetime.datetime.today()
    print("List Of Holidays:\n")
    for hols in dict_holidays:
        if(curr_date <= hols['FullDate']):
            if(hols['Type'] == holiday_type):
                hols_list.append(hols)
              
    sorted_hols = sorted(hols_list, key=lambda x: x['FullDate'])
    print("\n")
    print(sorted_hols) 
    return

holiday_type_func("Federal Holiday")

List Of Holidays:



[{'Day': 'Monday', 'Name': 'Labor Day', 'Type': 'Federal Holiday', 'Details': nan, 'Year': 2019, 'Dates': '2', 'Month': 'Sep', 'FullDate': Timestamp('2019-09-02 00:00:00')}, {'Day': 'Monday', 'Name': 'Columbus Day', 'Type': 'Federal Holiday', 'Details': 'All except AK,AR,CA,DE,FL,HI,MI,MN,ND,NM,NV,OR,SD,TX,VT,WA,WY,', 'Year': 2019, 'Dates': '14', 'Month': 'Oct', 'FullDate': Timestamp('2019-10-14 00:00:00')}, {'Day': 'Monday', 'Name': 'Veterans Day', 'Type': 'Federal Holiday', 'Details': nan, 'Year': 2019, 'Dates': '11', 'Month': 'Nov', 'FullDate': Timestamp('2019-11-11 00:00:00')}, {'Day': 'Thursday', 'Name': 'Thanksgiving Day', 'Type': 'Federal Holiday', 'Details': nan, 'Year': 2019, 'Dates': '28', 'Month': 'Nov', 'FullDate': Timestamp('2019-11-28 00:00:00')}, {'Day': 'Wednesday', 'Name': 'Christmas Day', 'Type': 'Federal Holiday', 'Details': nan, 'Year': 2019, 'Dates': '25', 'Month': 'Dec', 'FullDate': Timestamp('2019-12-25 00:00:00')}]
